In [1]:
import xgboost as xgb

In [4]:
from xgboost import XGBRegressor
X = [[.1, 0], [.1, 1], [.3, 0], [.3, 1],]*4
y = [1, 2, 3, 4,]*4

regressor = XGBRegressor(n_estimators=2, learning_rate=1.0, subsample=1.0, base_score=0)
regressor.fit(X, y)
print(regressor.predict(X[:4]), '\n')
for line in regressor.get_booster().get_dump(): 
    print(line)

[1.0666666 2.0246913 3.0222223 3.8024693] 

0:[f0<0.2] yes=1,no=2,missing=1
	1:leaf=1.33333
	2:leaf=3.11111

0:[f1<-9.53674e-07] yes=1,no=2,missing=1
	1:[f0<0.2] yes=3,no=4,missing=3
		3:leaf=-0.266667
		4:leaf=-0.0888889
	2:leaf=0.691358



In [6]:
booster = regressor.get_booster()

In [70]:
out = booster.get_dump(dump_format="json", with_stats=False)
tt = []
for i in out:
    tt.append(eval(i))
tt

[{'children': [{'leaf': 1.33333, 'nodeid': 1}, {'leaf': 3.11111, 'nodeid': 2}],
  'depth': 0,
  'missing': 1,
  'no': 2,
  'nodeid': 0,
  'split': 'f0',
  'split_condition': 0.2,
  'yes': 1},
 {'children': [{'children': [{'leaf': -0.266667, 'nodeid': 3},
     {'leaf': -0.0888889, 'nodeid': 4}],
    'depth': 1,
    'missing': 3,
    'no': 4,
    'nodeid': 1,
    'split': 'f0',
    'split_condition': 0.2,
    'yes': 3},
   {'leaf': 0.691358, 'nodeid': 2}],
  'depth': 0,
  'missing': 1,
  'no': 2,
  'nodeid': 0,
  'split': 'f1',
  'split_condition': -9.53674e-07,
  'yes': 1}]

In [65]:
tt = []
for i in out:
    tt.append(eval(i))
tt

[{'children': [{'cover': 8, 'leaf': 1.33333, 'nodeid': 1},
   {'cover': 8, 'leaf': 3.11111, 'nodeid': 2}],
  'cover': 16,
  'depth': 0,
  'gain': 8.99347,
  'missing': 1,
  'no': 2,
  'nodeid': 0,
  'split': 'f0',
  'split_condition': 0.2,
  'yes': 1},
 {'children': [{'children': [{'cover': 4, 'leaf': -0.266667, 'nodeid': 3},
     {'cover': 4, 'leaf': -0.0888889, 'nodeid': 4}],
    'cover': 8,
    'depth': 1,
    'gain': 0.0438957,
    'missing': 3,
    'no': 4,
    'nodeid': 1,
    'split': 'f0',
    'split_condition': 0.2,
    'yes': 3},
   {'cover': 8, 'leaf': 0.691358, 'nodeid': 2}],
  'cover': 16,
  'depth': 0,
  'gain': 3.491,
  'missing': 1,
  'no': 2,
  'nodeid': 0,
  'split': 'f1',
  'split_condition': -9.53674e-07,
  'yes': 1}]

In [21]:
import coremltools

In [58]:
from coremltools.models.tree_ensemble import TreeEnsembleRegressor as _TreeEnsembleRegressor

from coremltools._deps import HAS_XGBOOST as _HAS_XGBOOST

from coremltools.converters.xgboost._tree_ensemble import recurse_json

import numpy as _np

if _HAS_XGBOOST:
    import xgboost as _xgboost

def my_convert_tree_ensemble(model, feature_names = None, target = 'target', force_32bit_float = True):
    """Convert a generic tree model to the protobuf spec.
    This currently supports:
      * Decision tree regression
    Parameters
    ----------
    model: str | Booster
        Path on disk where the XGboost JSON representation of the model is or
        a handle to the XGboost model.
    feature_names : list of strings or None
        Names of each of the features. When set to None, the feature names are
        extracted from the model.
    target: str,
        Name of the output column.
    force_32bit_float: bool
        If True, then the resulting CoreML model will use 32 bit floats internally.
    Returns
    -------
    model_spec: An object of type Model_pb.
        Protobuf representation of the model
    """
    if not(_HAS_XGBOOST):
        raise RuntimeError('xgboost not found. xgboost conversion API is disabled.')
    
    import json
    import os
    feature_map = None
    if isinstance(model,  (_xgboost.core.Booster, _xgboost.XGBRegressor)):

        # Testing a few corner cases that we don't support
        if isinstance(model, _xgboost.XGBRegressor):
            try:
                objective = model.get_xgb_params()["objective"]
            except:
                objective = None
            if objective in ["reg:gamma", "reg:tweedie"]:
                raise ValueError("Regression objective '%s' not supported for export." % objective)

        # Now use the booster API.
        if isinstance(model, _xgboost.XGBRegressor):
            # Name change in 0.7
            if hasattr(model, 'get_booster'):
                model = model.get_booster()
            else:
                model = model.booster()

        # Xgboost sometimes has feature names in there. Sometimes does not.
        if (feature_names is None) and (model.feature_names is None):
            raise ValueError("Feature names not present in the model. Must be provided during conversion.")
            feature_names = model.feature_names
        if feature_names is None:
            feature_names = model.feature_names

        xgb_model_str = model.get_dump(with_stats=True, dump_format = 'json')

        if model.feature_names:
            feature_map = {f:i for i,f in enumerate(model.feature_names)}

    # Path on the file system where the XGboost model exists.
    elif isinstance(model, str):
        if not os.path.exists(model):
            raise TypeError("Invalid path %s." % model)
        with open(model) as f:
            xgb_model_str = json.load(f)
        feature_map = {f:i for i,f in enumerate(feature_names)}
    else:
        raise TypeError("Unexpected type. Expecting XGBoost model.")

    mlkit_tree = _TreeEnsembleRegressor(feature_names, target)
    mlkit_tree.set_default_prediction_value(0.5)
    for xgb_tree_id, xgb_tree_str in enumerate(xgb_model_str):
        xgb_tree_json = json.loads(xgb_tree_str)
        recurse_json(mlkit_tree, xgb_tree_json, xgb_tree_id, node_id = 0,
                feature_map = feature_map, force_32bit_float = force_32bit_float)

    return mlkit_tree

In [59]:
core_ml_model2 = my_convert_tree_ensemble(model=booster)

In [63]:
core_ml_model2.tree_parameters

nodes {
  nodeBehavior: BranchOnValueLessThan
  branchFeatureValue: 0.20000000298023224
  trueChildNodeId: 1
  falseChildNodeId: 2
  missingValueTracksTrueChild: true
  relativeHitRate: 16.0
}
nodes {
  nodeId: 1
  nodeBehavior: LeafNode
  evaluationInfo {
    evaluationValue: 1.3333300352096558
  }
  relativeHitRate: 8.0
}
nodes {
  nodeId: 2
  nodeBehavior: LeafNode
  evaluationInfo {
    evaluationValue: 3.111109972000122
  }
  relativeHitRate: 8.0
}
nodes {
  treeId: 1
  nodeBehavior: BranchOnValueLessThan
  branchFeatureIndex: 1
  branchFeatureValue: -9.536739753457368e-07
  trueChildNodeId: 1
  falseChildNodeId: 2
  missingValueTracksTrueChild: true
  relativeHitRate: 16.0
}
nodes {
  treeId: 1
  nodeId: 1
  nodeBehavior: BranchOnValueLessThan
  branchFeatureValue: 0.20000000298023224
  trueChildNodeId: 3
  falseChildNodeId: 4
  missingValueTracksTrueChild: true
  relativeHitRate: 8.0
}
nodes {
  treeId: 1
  nodeId: 3
  nodeBehavior: LeafNode
  evaluationInfo {
    evaluationValu

In [22]:
core_ml_model = coremltools.converters.xgboost.convert(model=booster)

In [47]:
core_ml_model.__class__

coremltools.models.model.MLModel

In [49]:
core_ml_model.

TypeError: 'str' object is not callable

In [33]:
spec = core_ml_model.get_spec()

In [46]:
spec.__class__

Model_pb2.Model

In [43]:
spec_str = str(spec)

In [45]:
spec.

specificationVersion: 1
description {
  input {
    name: "f0"
    type {
      doubleType {
      }
    }
  }
  input {
    name: "f1"
    type {
      doubleType {
      }
    }
  }
  output {
    name: "target"
    type {
      doubleType {
      }
    }
  }
  predictedFeatureName: "target"
}
treeEnsembleRegressor {
  treeEnsemble {
    nodes {
      nodeBehavior: BranchOnValueLessThan
      branchFeatureValue: 0.20000000298023224
      trueChildNodeId: 1
      falseChildNodeId: 2
      missingValueTracksTrueChild: true
      relativeHitRate: 16.0
    }
    nodes {
      nodeId: 1
      nodeBehavior: LeafNode
      evaluationInfo {
        evaluationValue: 1.3333300352096558
      }
      relativeHitRate: 8.0
    }
    nodes {
      nodeId: 2
      nodeBehavior: LeafNode
      evaluationInfo {
        evaluationValue: 3.111109972000122
      }
      relativeHitRate: 8.0
    }
    nodes {
      treeId: 1
      nodeBehavior: BranchOnValueLessThan
      branchFeatureIndex: 1
      bran

In [44]:
spec_str

'specificationVersion: 1\ndescription {\n  input {\n    name: "f0"\n    type {\n      doubleType {\n      }\n    }\n  }\n  input {\n    name: "f1"\n    type {\n      doubleType {\n      }\n    }\n  }\n  output {\n    name: "target"\n    type {\n      doubleType {\n      }\n    }\n  }\n  predictedFeatureName: "target"\n}\ntreeEnsembleRegressor {\n  treeEnsemble {\n    nodes {\n      nodeBehavior: BranchOnValueLessThan\n      branchFeatureValue: 0.20000000298023224\n      trueChildNodeId: 1\n      falseChildNodeId: 2\n      missingValueTracksTrueChild: true\n      relativeHitRate: 16.0\n    }\n    nodes {\n      nodeId: 1\n      nodeBehavior: LeafNode\n      evaluationInfo {\n        evaluationValue: 1.3333300352096558\n      }\n      relativeHitRate: 8.0\n    }\n    nodes {\n      nodeId: 2\n      nodeBehavior: LeafNode\n      evaluationInfo {\n        evaluationValue: 3.111109972000122\n      }\n      relativeHitRate: 8.0\n    }\n    nodes {\n      treeId: 1\n      nodeBehavior: Branch